In [6]:
# import necessary libraries
from datetime import date, datetime, timedelta
import mysql.connector
from mysql.connector import errorcode
import pandas as pd

DB_NAME = 'timperio'
USERNAME = 'root'
PASSWORD = 'password'
HOST = 'localhost'

# connect to the database
try:
    connection = mysql.connector.connect(host=HOST, user=USERNAME, password=PASSWORD, database=DB_NAME)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your username/password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
        
cursor = connection.cursor()

In [7]:
# Load the data from Excel and convert the date format
df_sales = pd.read_csv('CSV_Sales_Data_updated_Sep 2023.csv')
# df_sales['Sale Date'] = pd.to_datetime(df_sales['Sale Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
df_sales = df_sales.replace({float("NaN"): None})

In [8]:
# Insert the data into the database
# Have to insert data in this order: Customer, Product, Sale

# Insert data into Customer table
try:
    for index, row in df_sales.iterrows():
        sql = """
        INSERT INTO customers (customer_id, zipcode) VALUES (%s, %s)
        ON DUPLICATE KEY UPDATE `zipcode` = VALUES (zipcode)
        """
        
        values = (
            row['Customer ID'],
            row['ZipCode']
        )
        cursor.execute(sql, values)
        connection.commit()
except mysql.connector.Error as err:
    print(err)

1062 (23000): Duplicate entry '2' for key 'customers.PRIMARY'


In [9]:
# close cursor and connection
cursor.close()
connection.close()